In [10]:
import os
import sys
sys.path.append('./src')
from re import I
from BioSeq2Seq import SamplePreprocess, BioSeq2Seq
import numpy as np

(1) data

In [ ]:
chr_length = {'chr1':249250621, 'chr2':243199373, 'chr3':198022430, 'chr4':191154276, 'chr5':180915260, 'chr6':171115067,
            'chr7':159138663, 'chrX':155270560, 'chr8':146364022, 'chr9':141213431, 'chr10':135534747, 'chr11':135006516,
            'chr12':133851895, 'chr13':115169878, 'chr14':107349540, 'chr15':102531392, 'chr16':90354753, 'chr17':81195210,
            'chr18':78077248, 'chr20':63025520, 'chrY':59373566, 'chr19':59128983, 'chr22':51304566, 'chr21':48129895,
            }

reference_genome_file = 'hg19.fa'
input_data_peak_path = ['ropeak1.bed',
                        'ropeak2.bed',
                        ]

sequence_data_file = [
                        [
                        ['roseq/G1_minus.bw', 'roseq/G1_plus.bw'],
                        ['roseq/G2_minus.bw', 'roseq/G2_plus.bw'],
                        ]
                        
                        ]


init_lr=0.00001

include_chr = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
              'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17',
              'chr18', 'chr19', 'chr20', 'chrX']

blacklist_file = 'genome_regions/blacklist/hg19Blacklist_1000k.bed'

save_path = 'test_samples/TF/1.5_multi_trunk_90TF_frezzing_1.2'

(2) target file

In [11]:
output_data_peak_path = ['ouput_peaks_dir']

target_list = ['H3K4me1', 'H3K4me2', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K9ac', 'H3K9me3', 'H4K20me1']


target_seq_file = ['H3K4me1.bw', 'H3K4me2.bw', 'H3K4me3.bw', 'H3K27ac.bw', 'H3K27me3.bw', 'H3K36me3.bw', 'H3K9ac.bw', 'H3K9me3.bw', 'H4K20me1.bw']


(3) sample preprocess

In [ ]:
sample_preprocess = SamplePreprocess(reference_genome_file=reference_genome_file,
                                sequencing_data_file=sequence_data_file,
                                blacklist_file=blacklist_file,
                                include_chr=include_chr)


candidate_regions = sample_preprocess.get_candidate_regions()

samples = sample_preprocess.get_samples(seq_length=114688,
                                        overlap=81920,
                                        peak_paths_input_data=input_data_peak_path,
                                        peak_paths_output_data=output_data_peak_path
                                        )

train_samples = samples

validation_samples = sample_preprocess.get_evaluation_samples(include_chr=['chr21'], blacklist_file=blacklist_file)

(4) build model

In [7]:
tfModel = BioSeq2Seq(reference_genome_file,
                  sequencing_data_file=sequence_data_file,
                  target_sequencing_file=target_seq_file)

model = tfModel.build_model(target_list, nan=0, init_lr=init_lr, channels=576, num_transformer_layers=9)

(5) training

In [8]:
tfModel.train(train_samples,
        validation_samples,
        epoch_num=2,
        step_per_epoch=50,
        evaluation_epoch_num=1,
        valid_max_steps=1000000,
        save_path=save_path,    
        lr_attenuation=1.5,
        lr_trans_epoch=10,
        )

epoch:  0
lr:  1e-05


100%|██████████| 50/50 [03:41<00:00,  4.43s/it]  
428it [08:05,  1.13s/it]


[0.12500747 0.1223126  0.16571791 0.23384675 0.18541676 0.19872913
 0.23052779 0.13684276 0.28757748 0.1745195  0.20210668 0.17373355
 0.21022968 0.2367971  0.10804538 0.09303424 0.09122732 0.18305336
 0.12837936 0.23446873 0.13253114 0.15971035 0.17954937 0.11865442
 0.16155466 0.18872721 0.22465523 0.26433748 0.2556613  0.11839192
 0.24053393 0.24728997 0.2062743  0.26479772 0.22631073 0.198558
 0.21578112 0.08165576 0.19164635 0.18605278 0.12225598 0.1276341
 0.11952565 0.24692623 0.10100731 0.12004413 0.18907121 0.16126965
 0.10750887 0.1285111  0.26487654 0.3407516  0.2088276  0.2611104
 0.17490652 0.10487232 0.29130238 0.17553286 0.13183148 0.11613179
 0.20371294 0.18050689 0.1656324  0.1347544  0.13940893 0.22423562
 0.17014879 0.00970901 0.19556816 0.10203109 0.2079532  0.16519494
 0.20932308 0.13623111 0.05549532 0.08526751 0.05360943 0.09570992
 0.07631708 0.1688324  0.07313491 0.12346239 0.09147055 0.13000065
 0.15821922 0.09377877 0.16048712 0.11224625 0.09839149 0.12756829

100%|██████████| 50/50 [02:43<00:00,  3.27s/it] 
428it [08:02,  1.13s/it]


[0.13592201 0.12774833 0.17020565 0.24143265 0.18177429 0.23421054
 0.21592206 0.12712441 0.2614206  0.21287502 0.1756186  0.2122699
 0.2234968  0.2623338  0.12433914 0.1181268  0.08920158 0.19885603
 0.15047546 0.24504547 0.16187324 0.15417142 0.203633   0.1354475
 0.19075434 0.19320595 0.23921905 0.277062   0.2718544  0.13581966
 0.22565006 0.2672075  0.2078669  0.25802052 0.2356157  0.2135637
 0.23090722 0.10341426 0.22201894 0.19500354 0.16443197 0.1500288
 0.13406779 0.26065162 0.1177911  0.1365334  0.20401856 0.18480548
 0.1107014  0.2067971  0.3081758  0.32458338 0.23127961 0.2873538
 0.20210338 0.10176359 0.30323827 0.19433996 0.13673285 0.12023965
 0.212283   0.19276543 0.2084805  0.17951472 0.15625668 0.2138051
 0.17593603 0.02713221 0.22367398 0.11064563 0.19481145 0.18546578
 0.22157554 0.1401228  0.06111424 0.08746131 0.06029386 0.09712636
 0.08166897 0.17710178 0.07363794 0.1273617  0.0957075  0.1365242
 0.16571139 0.0951021  0.1677172  0.11612512 0.10198816 0.13565768]


(<TFBS.model_TFBS.TFModel at 0x7ffa18241b50>,
 [0.13690318185836076, 0.006062163086025976])

(6) load trained model

In [ ]:
tfModel.load_weights('saved_model/best_model')

(7) predict to file.bw

In [ ]:
outpath = 'out/'

reference_genome_file = 'hg19.fa'

sequence_data_file = [
                        [
                        ['roseq/G1_minus.bw', 'roseq/G1_plus.bw'],
                        ]
                        
                        ]

blacklist_file = 'genome_regions/blacklist/hg19Blacklist_1000k.bed'

sample_preprocess = SamplePreprocess(reference_genome_file=reference_genome_file,
                                sequencing_data_file=sequence_data_file,
                                blacklist_file=blacklist_file,
                                include_chr=include_chr)

candidate_regions = sample_preprocess.get_candidate_regions()

test_samples = sample_preprocess.get_evaluation_samples(include_chr=['chr22'], blacklist_file=blacklist_file)
tfModel.predict(test_samples,
            out_path=outpath,
            reference_genome_file=reference_genome_file,
            sequencing_data_file=sequence_data_file)